# 1. Simulations and design using the finite element method

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
x = np.array([0, 0, 2, 0, 4, -1])
F = np.array([0, 0, 0, 0, 0, -19])


# 2. Strategy to solve KU = P

In [2]:
def reorder_rows_columns(K,equation_numbers):
    # construct the matrix Khat
     x = len(equation_numbers) 
    y = equation_numbers - 1
    Khat = K.copy()
    for i in range(x):
        Khat[i] = K[y[i]]
        
    Ktemp = Khat.copy()
    for i in range(x):
        for j in range(x):
            Khat[i][j] = Ktemp[i][y[j]]
    return Khat


In [3]:
def partition_stiffness_matrix(K,equation_numbers,nk):
    # construct the smaller matrices
    k = reorder_rows_columns(K, equation_numbers)
    Kff = k[nk:, nk:]
    Kfp = k[nk:, 0:nk]
    Kpf = k[0:nk, nk:]
    Kpp = k[0:nk, 0:nk]
    return  Kpp,Kpf,Kfp,Kff

Kpp,Kpf,Kfp,Kff = partition_stiffness_matrix(K,equation_numbers,3)
xp = np.array([0., 0., 0.])
Ff = np.array([0., 0., -10.])

def fea_solve(Kpp,Kpf,Kfp,Kff,xp,Ff):
    # do stuff here
    tmp = Ff - Kfp@xp
    xf = la.solve(Kff,tmp)
    Fp = Kpp@xp + Kpf@xf
    return xf,Fp
 
Kpp, Kpf, Kfp, Kff = partition_stiffness_matrix(K,equation_numbers, 3)

xp = np.array([0, 0, 0],dtype = 'float')
Ff = np.array([0, 0, -10],dtype = 'float')

xf, Fp = fea_solve(Kpp,Kpf,Kfp,Kff,xp,Ff)

image_xf = plot_truss(xf)


# 3. LU and Triangular Solve

In [4]:
def my_lu(A):
    # LU Factorization
    M = A.copy()
    for i in range(A.shape[0]-1):
        for j in range(i+1,A.shape[0]):
            M[j, i] = M[j, i] / M[i, i]
            for k in range(i+1,A.shape[0]):
                M[j,k] -= M[i,k]*M[j,i]
    return M


In [5]:
def my_triangular_solve(M, b):
    # b: 1d array with the right-hand of the linear system of equations
    # implement Forward and Backward substitution
    #Forward - substitution - solve Ly  = b for y
     x = np.zeros(b.shape[0])
    y = np.zeros(b.shape[0])
    z = np.zeros((b.shape[0] ,b.shape[0]))
    for i in range(b.shape[0]):
        z[i, i] = 1
        for j in range(i):
            z[i,j] = M[i,j]
    for i in range(b.shape[0]):
        t = b[i]
        for j in range(i):
            t -= y[j] * z[i,j]
        y[i] = t/z[i,i]
    for i in range(b.shape[0]-1, -1, -1):
        t = y[i]
        for j in range(i, b.shape[0]):
            t -= x[j] * M[i,j]
        x[i] = t/M[i,i]
    return x


def fea_solve(Kpp, Kpf, Kfp, Kff, xp, Ff):
    # Use my_lu and my_triangular_solve

    xf = my_triangular_solve(my_lu(Kff), Ff - Kfp @ xp)
    Fp = Kpp @ xp + Kpf @ xf
    return xf, Fp


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.matmul(Kfp, xp)
y = my_lu(Kff)

xf_1 = my_triangular_solve(y, Ff[:, 0] - x)
image_xf_1 = plot_truss(xf_1)
xf_2 = my_triangular_solve(y, Ff[:, 1] - x)
image_xf_2 = plot_truss(xf_2)
